In [2]:
import pandas as pd

In [3]:
df = pd.read_excel('/home/copelco/Desktop/MD/2013.xlsx')

In [47]:
len(df.index)

760613

In [40]:
df.Agency.value_counts()

Maryland State Police                                           208301
Baltimore Police Department                                      80308
Baltimore County Police Department                               72651
Montgomery County Police Department                              54647
Maryland Transportation Authority Police                         52497
Prince George's County Police Department                         40956
Anne Arundel County Police Department                            38590
Howard County Police Department                                  27433
Frederick Police Department                                      19840
Ocean City Police Department                                     13703
Charles County Sheriff's Office                                   9246
Harford County Sheriff's Office                                   9240
University of Maryland College Park Police                        8194
Greenbelt Police Department                                       7024
Carrol

In [10]:
df.Race.unique()

array(['sero', 'warn', 'cit', 'arr', nan], dtype=object)

In [11]:
df.Outcome.unique()

array(['sero', 'warn', 'cit', 'arr', nan], dtype=object)

In [12]:
df.Search.unique()

array([nan, 'both', 'prop', 'pers', '   '], dtype=object)

In [13]:
df.Disposition.unique()

array([nan, 'none', 'contra', 'both', 'prop'], dtype=object)

In [45]:
df[df['Agency'].str.contains('Montgomery')]['Agency'].unique()

array(['Maryland National Capital Park Police Montgomery County',
       'Montgomery County Police Department',
       "Montgomery County Sheriff's Office"], dtype=object)

In [112]:
in_montgomery = df['Agency'] == 'Montgomery County Police Department'
missing_race = df['Race'].isnull()
did_search = df.Search.notnull()

In [106]:
len(df[in_montgomery].index)

54647

In [109]:
len(df[in_montgomery & missing_race].index)

13

In [110]:
df[in_montgomery].Race.value_counts()

w    21795
b    16802
h     9455
a     3617
o     2944
u       21
Name: Race, dtype: int64

In [111]:
df[in_montgomery].Race.value_counts().sum()

54634

In [71]:
df[in_montgomery].Search.count()

3053

In [113]:
df[in_montgomery].Search.count()/df[in_montgomery].Race.value_counts().sum()

0.055880953252553357

In [89]:
df[in_montgomery].Search.value_counts()

both    2141
prop     540
pers     372
Name: Search, dtype: int64

In [114]:
df[in_montgomery & did_search].Race.value_counts()

b    1204
w    1042
h     651
a      86
o      70
Name: Race, dtype: int64

In [115]:
df[in_montgomery & did_search].Race.value_counts().sum()

3053